[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1QPsU3xbgoY25jiMgoWamYhxKLT_iMHYA#scrollTo=XDspGgUS55Xo)

# Codice per montare GDrive su GColab

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd gdrive/MyDrive/ProjectAI/

/content/gdrive/.shortcut-targets-by-id/1A0bC_-pQ-oWUI2c5-EqQVAZ6051G0tKg/ProjectAI


In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import delle librerire fondamentali per l'analisi dei dati

In [4]:
import pandas as pd
import numpy as np
import re

# Utility per tipi e conversioni

In [24]:
from typing import List, Dict, Tuple
string_list = List[str]
one_hot_list = Tuple[int]
one_hot_encoding = Dict[str, one_hot_list]

## Mappe e funzioni per conversioni valori del dataset in valori numerici o booleani

In [25]:
def from_categorical_to_one_hot(categorical_data: string_list) -> one_hot_encoding:
    dictionary_to_return = {}
    indexes = range(len(categorical_data))
    for index, key in enumerate(categorical_data):
        dictionary_to_return[key] = tuple([1 if index == i else 0 for i in indexes])
    
    return dictionary_to_return

#Features categorica -> One Hot Encoding
list_MESI = ["Gennaio", "Febbraio", "Marzo", "Aprile", "Maggio", "Giugno", "Luglio", "Agosto", "Settembre", "Ottobre", "Novembre", "Dicembre", "Non disponibile"]
MESI = from_categorical_to_one_hot(list_MESI)

#Features categorica -> One Hot Encoding
list_ANNI = ["2000", "2001", "1999", "1998", "<=1997", ">=2002", "Non disponibile"]
ANNI = from_categorical_to_one_hot(list_ANNI)

def convert_question_result(result: str):
    return result == "Corretta" # alternativamente result == "Errata"

#Features categorica -> One Hot Encoding
list_REGOLARITA = ['Regolare', 'Posticipatario', 'Anticipatario', 'Dato mancante']
REGOLARITA = from_categorical_to_one_hot(list_REGOLARITA)

#Features categorica -> One Hot Encoding
list_AREA_GEOGRAFICA_5_ISTAT = ['Sud', 'Nord est', 'Centro', 'Nord ovest', 'Isole']
AREA_GEOGRAFICA_5_ISTAT = from_categorical_to_one_hot(list_AREA_GEOGRAFICA_5_ISTAT)

#Features categorica -> One Hot Encoding
list_AREA_GEOGRAFICA_5 = ['Sud', 'Nord est', 'Centro', 'Nord ovest', 'Sud e isole']
AREA_GEOGRAFICA_5 = from_categorical_to_one_hot(list_AREA_GEOGRAFICA_5)

#Features categorica -> One Hot Encoding
list_AREA_GEOGRAFICA_4 = ['Mezzogiorno', 'Nord est', 'Centro', 'Nord ovest']
AREA_GEOGRAFICA_4 = from_categorical_to_one_hot(list_AREA_GEOGRAFICA_4)

#Features categorica -> One Hot Encoding
list_AREA_GEOGRAFICA_3 = ['Mezzogiorno', 'Nord', 'Centro']
AREA_GEOGRAFICA_3 = from_categorical_to_one_hot(list_AREA_GEOGRAFICA_3)

#Features categorica -> One Hot Encoding
list_REGIONI = ['Campania', 'Emilia-Romagna', 'Lazio', 'Piemonte', 'Puglia', 'Lombardia', 'Veneto', 'Sicilia', 'Prov. Aut. Trento', 'Friuli-Venezia Giulia', 'Abruzzo', 'Liguria', 'Toscana', 'Sardegna', 'Calabria', 'Molise', 'Marche', 'Umbria', 'Basilicata', 'Prov. Aut. Bolzano (l. it.)']
REGIONI = from_categorical_to_one_hot(list_REGIONI)

#Features categorica -> One Hot Encoding
list_PROVINCE = ['', 'RE', 'FR', 'TO', 'BA', 'CO', 'LE', 'RO', 'CT', 'RM', 'TA', 'BS', 'SA', 'TN', 'UD', 'FG', 'LT', 'AG', 'CH', 'PC', 'TS', 'SR', 'SP', 'PD', 'SI', 'PA', 'TP', 'BO', 'CA', 'CN', 'RC', 'TE', 'MI', 'LC', 'LU', 'FI', 'AQ', 'TV', 'RG', 'VA', 'GO', 'MO', 'GE', 'AL', 'CB', 'PR', 'OR', 'VE', 'MC', 'NO', 'PT', 'MN', 'VR', 'PI', 'AP', 'LO', 'VI', 'SV', 'PU', 'BG', 'AR', 'VT', 'LI', 'SS', 'BR', 'RA', 'TR', 'SO', 'IM', 'PZ', 'GR', 'AN', 'PN', 'ME', 'CR', 'FE', 'BI', 'PV', 'PG', 'VB', 'BL', 'PE', 'CS', 'CZ', 'AV', 'RN', 'CL', 'AT', 'MS', 'KR', 'RI', 'EN', 'CE', 'MT', 'VV', 'VC', 'NU', 'FC', 'PO', 'BZ', 'BN', 'IS', 
            'NA', # presente in cod_provincia_ISTAT ma non in sigla_provincia_istat
            'PS', # presente in cod_provincia_ISTAT ma non in sigla_provincia_istat
            'FO', # presente in cod_provincia_ISTAT ma non in sigla_provincia_istat
            'LB', # presente in cod_provincia_ISTAT ma non in sigla_provincia_istat
]
PROVINCE = from_categorical_to_one_hot(list_PROVINCE)

#Features categorica -> One Hot Encoding
list_CITTADINANZA = ['Italiano', 'Straniero II generazione', 'Straniero I generazione', 'Dato mancante']
CITTADINANZA = from_categorical_to_one_hot(list_CITTADINANZA)

list_VOTI_NUMERICI = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
def voto_orale_decode(voto_orale: str):
    if voto_orale in list_VOTI_NUMERICI:
        return int(voto_orale)
    elif voto_orale == 'Non disponibile':
        return np.nan
    elif voto_orale == 'Non classificato': 
        return 0

list_VOTI_NAN = ['Non disponibile', 'Senza voto scritto']
def voto_scritto_decode(voto_scritto: str):
    if voto_scritto in list_VOTI_NUMERICI:
        return int(voto_scritto)
    elif voto_scritto in list_VOTI_NAN:
        return np.nan
    elif voto_scritto == 'Non classificato': 
        return 0
    
#Features categorica -> One Hot Encoding
list_PROFESSIONI = ['1. Disoccupato/a', '2. Casalingo/a', '3. Dirigente, docente universitario, funzionario o ufficiale militare', '4. Imprenditore/proprietario agricolo', '5. Professionista dipendente, sottuff. militare o libero profession. (medico, av', '6. Lavoratore in proprio (commerciante, coltivatore diretto, artigiano, meccanic', '7. Insegnante, impiegato, militare graduato', '8. Operaio, addetto ai servizi/socio di cooperativa', '9. Pensionato/a', '10. Non disponibile']
PROFESSIONI = from_categorical_to_one_hot(list_PROFESSIONI)

#Features categorica -> One Hot Encoding
list_TITOLI = ['1. Licenza elementare', '2. Licenza media', '3. Qualifica professionale triennale', '4. Diploma di maturità', '5. Altro titolo di studio superiore al diploma (I.S.E.F., Accademia di Belle Art', '6. Laurea o titolo superiore (ad esempio Dottorato di Ricerca)', '7. Non disponibile']
TITOLI = from_categorical_to_one_hot(list_TITOLI)

#Features categorica -> One Hot Encoding
list_LUOGHI_GENITORI = ['Italia (o Repubblica di San Marino)', 'Unione Europea', 'Paese europeo Non UE', 'Altro', 'Non disponibile']
LUOGHI_GENITORI = from_categorical_to_one_hot(list_LUOGHI_GENITORI)

#Features categorica -> One Hot Encoding
list_FREQUENZA_SCUOLA = ['No', 'Sì', 'Non disponibile']
FREQUENZA_SCUOLA = from_categorical_to_one_hot(list_FREQUENZA_SCUOLA)

#Features categorica -> One Hot Encoding
list_LUOGO_DI_NASCITA = ['Italia (o Repubblica di San Marino)', 'Unione Europea', 'Paese europeo Non UE', 'Altro', 'Non disponibile']
LUOGO_DI_NASCITA = from_categorical_to_one_hot(list_LUOGO_DI_NASCITA)

## Gestione conversione valori del dataset in tipi più facili da utilizzare

In [26]:
columns_converters = {
    "CODICE_SCUOLA": str, #identificativo della scuola
    "CODICE_PLESSO": str, #identificativo del plesso
    "CODICE_CLASSE": str, #identificato della classe
    "macrotipologia": str, #categoria di scuola
    "campione": int, #campione di riferimento
    "livello": int,
    "prog": int,
    "CODICE_STUDENTE": str, #codice dello studente
    "sesso": str, #sesso dello studente
    "mese": lambda month: MESI[month], #mese di nascita
    "anno": lambda year: ANNI[year], #anno di nascita
    "luogo": lambda luogo: LUOGO_DI_NASCITA[luogo],
    "eta": str, # cosa vuol dire eta?
    "codice_orario": lambda _: np.nan, # unico dato: Mancante di sistema
    "freq_asilo_nido": lambda frequenza: FREQUENZA_SCUOLA[frequenza],
    "freq_scuola_materna": lambda frequenza: FREQUENZA_SCUOLA[frequenza],
    "luogo_padre": lambda luogo: LUOGHI_GENITORI[luogo],
    "titolo_padre": lambda titolo: TITOLI[titolo],
    "prof_padre": lambda professione: PROFESSIONI[professione],
    "luogo_madre": lambda luogo: LUOGHI_GENITORI[luogo],
    "titolo_madre": lambda titolo: TITOLI[titolo],
    "prof_madre": lambda professione: PROFESSIONI[professione],
    "voto_scritto_ita": lambda voto: voto_scritto_decode(voto),
    "voto_orale_ita": lambda voto: voto_orale_decode(voto),
    "voto_scritto_mat": lambda voto: voto_scritto_decode(voto),
    "voto_orale_mat": lambda voto: voto_orale_decode(voto),
    "D1": lambda result: convert_question_result(result),
    "D2": lambda result: convert_question_result(result),
    "D3_a": lambda result: convert_question_result(result),
    "D3_b": lambda result: convert_question_result(result),
    "D4_a": lambda result: convert_question_result(result),
    "D4_b": lambda result: convert_question_result(result),
    "D4_c": lambda result: convert_question_result(result),
    "D4_d": lambda result: convert_question_result(result),
    "D5_a": lambda result: convert_question_result(result),
    "D5_b": lambda result: convert_question_result(result),
    "D6": lambda result: convert_question_result(result),
    "D7_a": lambda result: convert_question_result(result),
    "D7_b": lambda result: convert_question_result(result),
    "D8": lambda result: convert_question_result(result),
    "D9": lambda result: convert_question_result(result),
    "D10_a": lambda result: convert_question_result(result),
    "D10_b1": lambda result: convert_question_result(result),
    "D10_b2": lambda result: convert_question_result(result),
    "D10_b3": lambda result: convert_question_result(result),
    "D11_a": lambda result: convert_question_result(result),
    "D11_b": lambda result: convert_question_result(result),
    "D12_a": lambda result: convert_question_result(result),
    "D12_b": lambda result: convert_question_result(result),
    "D13_a": lambda result: convert_question_result(result),
    "D13_b": lambda result: convert_question_result(result),
    "D13_c": lambda result: convert_question_result(result),
    "D14": lambda result: convert_question_result(result),
    "D15": lambda result: convert_question_result(result),
    "D16_a": lambda result: convert_question_result(result),
    "D16_b": lambda result: convert_question_result(result),
    "D16_c": lambda result: convert_question_result(result),
    "D16_d": lambda result: convert_question_result(result),
    "D17_a": lambda result: convert_question_result(result),
    "D17_b": lambda result: convert_question_result(result),
    "D18": lambda result: convert_question_result(result),
    "D19_a": lambda result: convert_question_result(result),
    "D19_b": lambda result: convert_question_result(result),
    "D20": lambda result: convert_question_result(result),
    "D21": lambda result: convert_question_result(result),
    "D22": lambda result: convert_question_result(result),
    "D23_a": lambda result: convert_question_result(result),
    "D23_b": lambda result: convert_question_result(result),
    "D23_c": lambda result: convert_question_result(result),
    "D23_d": lambda result: convert_question_result(result),
    "D24_a": lambda result: convert_question_result(result),
    "D24_b": lambda result: convert_question_result(result),
    "D25": lambda result: convert_question_result(result),
    "D26_a": lambda result: convert_question_result(result),
    "D26_b": lambda result: convert_question_result(result),
    "D26_c": lambda result: convert_question_result(result),
    "D26_d": lambda result: convert_question_result(result),
    "regolarità": lambda regular: REGOLARITA[regular],
    "cittadinanza": lambda cittadinanza: CITTADINANZA[cittadinanza],
    "cod_provincia_ISTAT": lambda province_istat: PROVINCE[province_istat.upper()],
    "sigla_provincia_istat": lambda province: PROVINCE[province],
    "Nome_reg": lambda regione: REGIONI[" ".join(regione.split())],
    "Cod_reg": lambda codice: REGIONI[codice],
    "Areageo_3": lambda area: AREA_GEOGRAFICA_3[area],
    "Areageo_4": lambda area: AREA_GEOGRAFICA_4[area],
    "Areageo_5": lambda area: AREA_GEOGRAFICA_5[area],
    "Areageo_5_Istat": lambda area: AREA_GEOGRAFICA_5_ISTAT[area],
    "Pon": lambda pon: True if pon == "Area_Pon" else False, # lo studente appartiene all'aera Pon oppure no
    "pu_ma_gr": int,
    "pu_ma_no": float,
    "Fattore_correzione_new": float,
    "Cheating": float,
    "PesoClasse": lambda val: float(val) if val != "" else np.nan,
    "PesoScuola": lambda val: float(val) if val != "" else np.nan,
    "PesoTotale_Matematica": lambda val: float(val) if val != "" else np.nan,
    "WLE_MAT": float,
    "WLE_MAT_200": float,
    "WLE_MAT_200_CORR": float,
    "pu_ma_no_corr": float,
    "n_stud_prev": lambda val: int(float(val)),
    "n_classi_prev": lambda val: int(float(val)),
    "LIVELLI": int,
    "DROPOUT": bool,
}

# Lettura del dataset

In [27]:
dataset = pd.read_csv("Data/invalsi_mat_2014.csv", 
                      sep=';', 
                      converters=columns_converters)

# Matrici correlazione tra domande

In [29]:
questions_columns = [col for col in columns_converters.keys() if re.search("^D\d", col)]
questions_dataset = dataset[questions_columns]

## Standard correlation coefficient (pearson method)

In [30]:
questions_correlate_matrix_pearson = questions_dataset.corr(method='pearson')
questions_correlate_matrix_pearson.style.background_gradient(cmap='coolwarm')

## Kendall Tau correlation coefficient

In [31]:
questions_correlate_matrix_pearson = questions_dataset.corr(method='kendall')
questions_correlate_matrix_pearson.style.background_gradient(cmap='coolwarm')

## Spearman rank correlation

In [32]:
questions_correlate_matrix_pearson = questions_dataset.corr(method='spearman')
questions_correlate_matrix_pearson.style.background_gradient(cmap='coolwarm')

# Nota sulle matrici di correlazione ottenute
Le **matrici di correlazione sulle domande** non evidenziano significative dipendenze lineari tra di essse: i valori più elevati appaiono in corrispondenza di domande consecutive, il più delle volte parti della stessa domanda (e.g. D7, D3 e D4).  
Conseguentemente, **non è possibile realizzare alcuna riduzione di dimensionalità**.  
Di seguito, verifichiamo se anche filtrando per area geografica la stessa situazione si verifica.

## Standard correlation coefficient (pearson method) filtered by Areageo_3 ( == [0, 1, 0])

In [43]:
questions_dataset_areageo_3 = dataset.query('Areageo_3 == (0, 1, 0)')
print(questions_dataset_areageo_3['Areageo_3'])
questions_dataset_areageo_3 = questions_dataset_areageo_3[questions_columns]

Series([], Name: Areageo_3, dtype: object)


In [36]:
questions_correlate_matrix_pearson_areageo_3 = questions_dataset_areageo_3.corr(method='pearson')
questions_correlate_matrix_pearson_areageo_3.style.background_gradient(cmap='coolwarm')

/usr/local/lib/python3.7/dist-packages/pandas/io/formats/style.py:1126: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(s.to_numpy()) if vmin is None else vmin
/usr/local/lib/python3.7/dist-packages/pandas/io/formats/style.py:1127: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(s.to_numpy()) if vmax is None else vmax


,D1,D2,D3_a,D3_b,D4_a,D4_b,D4_c,D4_d,D5_a,D5_b,D6,D7_a,D7_b,D8,D9,D10_a,D10_b1,D10_b2,D10_b3,D11_a,D11_b,D12_a,D12_b,D13_a,D13_b,D13_c,D14,D15,D16_a,D16_b,D16_c,D16_d,D17_a,D17_b,D18,D19_a,D19_b,D20,D21,D22,D23_a,D23_b,D23_c,D23_d,D24_a,D24_b,D25,D26_a,D26_b,D26_c,D26_d
D1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
D2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
D3_a,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
D3_b,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
D4_a,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
D4_b,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
D4_c,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
D4_d,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
D5_a,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
D5_b,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


# ESEGUI QUEST'ULTIMA CELLA PER SMONTARE GDRIVE

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.
